This repo is for finding the theoretical/empirical error quickly

# Imports

In [30]:
import os
import numpy as np

from snap.wrapper import TorchWrapper
from snap.experiment import Experiment
from snap.nsd_data import get_neural_data
from snap.regression_utils_dd import regression_metric
import snap.models as models

import matplotlib.pyplot as plt

# Regression

## Set Important Variables

In [2]:
device = 'cuda'

Model args

In [3]:
modelNames = ['resnet50']
regionNames = ['V1']
activation_pooling = [None]
rand_proj_dim = None 
trained = True

Regression args

In [4]:
reg = np.logspace(-14, 6, 27).tolist() # alpha(s) to search through in ridge regression
num_trials = 1 # number of regresssions to run (they're averaged afterwards)

Data Loader args

In [5]:
num_samples = None # number of image:activity pairs to take from data

loader_kwargs = {'batch_size': 128,
                 'shuffle': False,
                 'num_workers': 4,
                 'pin_memory': True,
                }      

small_nsd_path = '/mnt/ceph/users/alargen/small_nsd/'

## Run the regression

In [12]:
def run_regression(model_name, data_loader_neural, images, labels, pooling):
    # Get the model
    model_kwargs = {'name': model_name,
                    'pretrained': trained,
                    'device': device}
    model, layers, identifier = models.get_model(**model_kwargs)
    model_wrapped = TorchWrapper(model,
                                layers=[], 
                                # layers=layers,
                                identifier=identifier,
                                activation_pooling=pooling)

    # Create the Experiment Class and pass additional metrics
    regression_kwargs = {'num_trials': num_trials, 
                        'reg': reg,
                        'num_points': 5, # this one doesn't do anything
                        }

    metric_fns = [regression_metric]
    exp = Experiment(model_wrapped,
                    metric_fns=metric_fns,
                    rand_proj_dim=rand_proj_dim)

    # Extract the activations of the layers passed above
    # using data_loader (only uses the inputs)
    exp.get_activations(data_loader_neural)

    # Compute metrics
    metric_kwargs = {'debug': False,
                    'epsilon': 1e-14
                    } | regression_kwargs

    exp_metrics = exp.compute_metrics(images=images,
                                        labels=labels,
                                        **metric_kwargs)
    return exp_metrics

In [13]:
for region in regionNames:
    # make the data loader
    data_loader_neural, images, labels = get_neural_data(region=region,
                                            loader_kwargs=loader_kwargs,
                                            data_path=small_nsd_path, num_samples=num_samples)
    
    for model_name in modelNames:
        print(f'\n\n\nAnalyzing {model_name}')
        for pooling in activation_pooling:
           exp_metrics = run_regression(model_name, data_loader_neural, 
                                        images, labels, pooling)
           
           # data visualization or some method of collecting the relevant data should go here

Shape of images: torch.Size([1000, 3, 224, 224])
Shape of brain responses: torch.Size([1000, 1110])



Analyzing resnet50
Getting layer activations...
Computing metrics...
Computing metrics for ['image_layer', 'response_layer']
{'image_layer': torch.Size([1000, 150528]), 'response_layer': torch.Size([1000, 1110])}
Computing spectrum...


Layer: 100%|██████████| 2/2 [00:00<00:00,  3.24it/s]


Computing regression_metric...


Layer:   0%|          | 0/2 [00:00<?, ?it/s]


 N: 150528, p: 600, Best Alpha: 1000000.0

 N: 150528, p: 800, Best Alpha: 1000000.0


...


 N: 150528, p: 600, Best Alpha: 1000000.0

 N: 150528, p: 800, Best Alpha: 1000000.0


Layer:  50%|█████     | 1/2 [01:48<01:48, 108.69s/it]


 N: 1110, p: 600, Best Alpha: 1e-14

 N: 1110, p: 800, Best Alpha: 1e-14

 N: 1110, p: 600, Best Alpha: 1e-14


Layer: 100%|██████████| 2/2 [03:26<00:00, 103.09s/it]


 N: 1110, p: 800, Best Alpha: 1e-14
Metric Computation completed!


In [22]:
exp_metrics['regression_metric']['uncent']['image_layer']['responses'].keys()

dict_keys(['pvals', 'P', 'N', 'C', 'cent', 'reg', 'gen_errs', 'tr_errs', 'test_errs', 'r2_gen', 'r2_tr', 'r2_test', 'pearson_tr', 'pearson_test', 'pearson_gen', 'gen_norm', 'tr_norm', 'test_norm', 'pvals_theory', 'kappa', 'gamma', 'eff_regs', 'mode_err_theory', 'gen_theory', 'tr_theory', 'r2_theory'])

In [42]:
exp_metrics['regression_metric']['uncent']['image_layer']['responses']['gen_errs'].shape 
# this is num_trials x pvals x num_voxels

(1, 2, 1110)